# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra
['/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-02-events.csv', '/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-17-events.csv', '/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-24-events.csv', '/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-29-events.csv', '/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-16-events.csv', '/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-12-events.csv', '/home/haseeb/Projects/Data-Engineering-Nano-Degree-Codes/Data Modelling with Apache Cassendra/event_data/2018-11-03-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         print((line[0], line[9], line[5], line[8], line[3]))
        session.execute(query, (line[0].replace("'", "''"), line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM music_app_history;"
try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')
Row(sessionid=23, iteminsession=5, artist="Soulja Boy Tell''em", length=201.1162872314453, song='Let Me Get Em')
Row(sessionid=23, iteminsession=6, artist='Bodo Wartke', length=645.2763061523438, song='Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃ\x83Â¶sisch_ Italienisch_ Spanisch_ HollÃ\x83Â¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessisch)')
Row(sessionid=23, iteminsession=7, artist='Evanescence', 

Row(sessionid=152, iteminsession=45, artist='Diplo', length=96.86158752441406, song='Florida')
Row(sessionid=152, iteminsession=46, artist='Afro Celt Sound System', length=425.01177978515625, song='Even In My Dreams')
Row(sessionid=152, iteminsession=47, artist='M.A. Numminen', length=166.5562744140625, song='TULENLIEKKI')
Row(sessionid=152, iteminsession=49, artist='Limp Bizkit', length=214.51710510253906, song="Rollin' (Air Raid Vehicle)")
Row(sessionid=152, iteminsession=50, artist='Downhere', length=230.7130126953125, song='In America (Album Version)')
Row(sessionid=152, iteminsession=51, artist='Roots Manuva', length=233.92608642578125, song='No Love')
Row(sessionid=152, iteminsession=52, artist='Lykke Li', length=162.19383239746094, song='Let It Fall')
Row(sessionid=152, iteminsession=53, artist='Plan B', length=222.82403564453125, song='Free')
Row(sessionid=152, iteminsession=54, artist='Octopus Project', length=498.3375244140625, song='Hypnopaedia')
Row(sessionid=152, iteminses

Row(sessionid=709, iteminsession=5, artist='TV On The Radio', length=275.87872314453125, song='Province')
Row(sessionid=709, iteminsession=6, artist='The Presets', length=271.8558654785156, song='Yippiyo-Ay')
Row(sessionid=487, iteminsession=0, artist='Copeland', length=279.326904296875, song='The Day I Lost My Voice (The Suitcase Song)')
Row(sessionid=938, iteminsession=1, artist='Alice Cooper', length=155.42811584472656, song='Under My Wheels (Live)')
Row(sessionid=938, iteminsession=2, artist='The Rolling Stones', length=271.49017333984375, song='Angie (1993 Digital Remaster)')
Row(sessionid=938, iteminsession=3, artist='Enrique Iglesias', length=264.2019958496094, song='Hero')
Row(sessionid=938, iteminsession=4, artist='Aphex Twin', length=547.1342163085938, song='Tha')
Row(sessionid=938, iteminsession=5, artist='Jack Johnson', length=283.9244384765625, song="They Do_ They Don't")
Row(sessionid=938, iteminsession=6, artist='Pixies', length=210.99057006835938, song='Hey')
Row(sessio

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "drop table if exists user_music_app_history; "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS user_music_app_history "
query = query + "(artist text, song text, userId int, userFirstName text, userLastName text, sessionId int, itemInSession int, PRIMARY KEY (userId, sessionId, itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_music_app_history (artist, song, userId, userFirstName, userLastName, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         print((line[0], line[9], line[5], line[8], line[3]))
        session.execute(query, (line[0].replace("'", "''"), line[9], int(line[10]), line[1], line[4], int(line[8]), int(line[3])))
    
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM user_music_app_history where userId=10 and sessionId=182;"
try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)


                    

Row(userid=10, sessionid=182, iteminsession=0, artist='Down To The Bone', song="Keep On Keepin' On", userfirstname='Sylvie', userlastname='Cruz')
Row(userid=10, sessionid=182, iteminsession=1, artist='Three Drives', song='Greece 2000', userfirstname='Sylvie', userlastname='Cruz')
Row(userid=10, sessionid=182, iteminsession=2, artist='Sebastien Tellier', song='Kilometer', userfirstname='Sylvie', userlastname='Cruz')
Row(userid=10, sessionid=182, iteminsession=3, artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', userfirstname='Sylvie', userlastname='Cruz')


In [15]:
# TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "drop table if exists user_songs_history; "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS user_songs_history "
query = query + "(song text, userFirstName text, userLastName text, PRIMARY KEY (song, userFirstName, userLastName));"
try:
    session.execute(query)
except Exception as e:
    print(e)

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_songs_history (song, userFirstName, userLastName)"
        query = query + "VALUES (%s, %s, %s);"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         print((line[0], line[9], line[5], line[8], line[3]))
        session.execute(query, (line[9], line[1], line[4]))
    
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT userFirstName, userLastName FROM user_music_app_history where song='All Hands Against His Own' ALLOW FILTERING;"
try:
    rows = session.execute(query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)


                    

Row(userfirstname='Sara', userlastname='Johnson')
Row(userfirstname='Tegan', userlastname='Levine')
Row(userfirstname='Jacqueline', userlastname='Lynch')


### Drop the tables before closing out the sessions

In [16]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table if exists user_songs_history; "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user_music_app_history; "
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user_songs_history; "
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()